In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os
from scipy.signal import find_peaks

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

input_dir = "My Training Data\Correct"
output_root = "split_reps/"
os.makedirs(output_root, exist_ok=True)

def extract_landmarks_and_frames(video_path):
    """Extract pose landmarks + keep raw frames for later video splitting."""
    cap = cv2.VideoCapture(video_path)
    sequence, frames = [], []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        if results.pose_landmarks:
            frame_landmarks = []
            for lm in results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z])
            sequence.append(frame_landmarks)
        else:
            # Append dummy frame to keep sync
            sequence.append([0]*99)
    cap.release()
    return np.array(sequence), frames

def get_motion_signal(sequence):
    """Choose the best joint (nose, shoulder, or hip) for motion tracking."""
    joints = [0, 11, 23]  # nose, left_shoulder, left_hip
    motion_signals = []
    for j in joints:
        y = sequence[:, j * 3 + 1]
        y_smooth = np.convolve(y, np.ones(5)/5, mode='same')
        motion_signals.append(y_smooth)
    ranges = [np.ptp(sig) for sig in motion_signals]
    best_idx = np.argmax(ranges)
    return motion_signals[best_idx]

def split_reps(sequence, signal, frames, save_dir, basename, fps):
    """Split both the numpy sequence and the original video frames."""
    minima, _ = find_peaks(-signal, distance=15)
    start = 0
    rep_count = 0

    for m in minima:
        end = m + 10
        rep_seq = sequence[start:end]
        rep_frames = frames[start:end]
        if len(rep_seq) > 20:  # ignore too short reps
            rep_count += 1

            # Save npy
            np.save(os.path.join(save_dir, f"{basename}_rep{rep_count}.npy"), rep_seq)

            # Save video segment
            h, w, _ = rep_frames[0].shape
            out_path = os.path.join(save_dir, f"{basename}_rep{rep_count}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
            for f in rep_frames:
                out.write(f)
            out.release()
        start = end
    return rep_count

for filename in os.listdir(input_dir):
    if not filename.endswith(".mp4"):
        continue

    print("Processing:", filename)
    video_path = os.path.join(input_dir, filename)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    sequence, frames = extract_landmarks_and_frames(video_path)
    if sequence.size == 0:
        print("  ⚠️ No pose detected, skipping:", filename)
        continue

    signal = get_motion_signal(sequence)

    name = os.path.splitext(filename)[0]
    save_dir = os.path.join(output_root, name)
    os.makedirs(save_dir, exist_ok=True)

    reps_found = split_reps(sequence, signal, frames, save_dir, name, fps)
    print(f"  ✅ {reps_found} reps saved to {save_dir}")

pose.close()
print("🎉 All videos processed successfully.")


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Processing: Pushup Vid 3.mp4
  ✅ 12 reps saved to split_reps/Pushup Vid 3
Processing: PushUps Vid 1.mp4
  ✅ 10 reps saved to split_reps/PushUps Vid 1
Processing: PushUps Vid 2.mp4
  ✅ 20 reps saved to split_reps/PushUps Vid 2
🎉 All videos processed successfully.


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from scipy.signal import find_peaks

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

input_dir = "My Training Data\Correct"
output_root = "split_reps_full/"
os.makedirs(output_root, exist_ok=True)

def extract_landmarks_and_frames(video_path):
    """Extract pose landmarks + keep raw frames for later video splitting."""
    cap = cv2.VideoCapture(video_path)
    sequence, frames = [], []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        if results.pose_landmarks:
            frame_landmarks = []
            for lm in results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z])
            sequence.append(frame_landmarks)
        else:
            # Append dummy frame to keep frame sync
            sequence.append([0]*99)
    cap.release()
    return np.array(sequence), frames

def get_motion_signal(sequence):
    """Choose the best joint (nose, shoulder, or hip) for motion tracking."""
    joints = [0, 11, 23]  # nose, left_shoulder, left_hip
    motion_signals = []
    for j in joints:
        y = sequence[:, j * 3 + 1]
        y_smooth = np.convolve(y, np.ones(7)/7, mode='same')
        motion_signals.append(y_smooth)
    ranges = [np.ptp(sig) for sig in motion_signals]
    best_idx = np.argmax(ranges)
    return motion_signals[best_idx]

def split_reps(sequence, signal, frames, save_dir, basename, fps):
    """Split the video into full reps (up → down → up)."""
    # Find peaks (top/up) and valleys (bottom/down)
    peaks, _ = find_peaks(signal, distance=15, prominence=0.01)
    valleys, _ = find_peaks(-signal, distance=15, prominence=0.01)

    # Combine and sort
    extrema = np.sort(np.concatenate([peaks, valleys]))
    
    rep_count = 0
    for i in range(len(extrema) - 1):
        start = extrema[i]
        end = extrema[i + 1]

        # Skip if interval too short
        if end - start < 10:
            continue

        # Detect "up → down → up" pattern
        if signal[start] > signal[end]:
            # This is the down motion (ignore)
            continue

        # One full rep between this peak and the next peak
        rep_seq = sequence[start:end]
        rep_frames = frames[start:end]
        rep_count += 1

        npy_path = os.path.join(save_dir, f"{basename}_rep{rep_count}.npy")
        np.save(npy_path, rep_seq)

        # Save video
        h, w, _ = rep_frames[0].shape
        out_path = os.path.join(save_dir, f"{basename}_rep{rep_count}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
        for f in rep_frames:
            out.write(f)
        out.release()

    return rep_count

for filename in os.listdir(input_dir):
    if not filename.endswith(".mp4"):
        continue

    print("Processing:", filename)
    video_path = os.path.join(input_dir, filename)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    sequence, frames = extract_landmarks_and_frames(video_path)
    if sequence.size == 0:
        print("  ⚠️ No pose detected, skipping:", filename)
        continue

    signal = get_motion_signal(sequence)

    name = os.path.splitext(filename)[0]
    save_dir = os.path.join(output_root, name)
    os.makedirs(save_dir, exist_ok=True)

    reps_found = split_reps(sequence, signal, frames, save_dir, name, fps)
    print(f"  ✅ {reps_found} full reps saved to {save_dir}")

pose.close()
print("🎉 All videos processed successfully.")


Processing: Pushup Vid 3.mp4
  ✅ 9 full reps saved to split_reps_full/Pushup Vid 3
Processing: PushUps Vid 1.mp4
  ✅ 10 full reps saved to split_reps_full/PushUps Vid 1
Processing: PushUps Vid 2.mp4
  ✅ 20 full reps saved to split_reps_full/PushUps Vid 2
🎉 All videos processed successfully.


In [6]:
#3rd try
import cv2
import mediapipe as mp
import numpy as np
import os
from scipy.signal import find_peaks

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)

input_dir = "My Training Data\Wrong"
output_root = "split_reps_full_Wrong/"
os.makedirs(output_root, exist_ok=True)

def extract_landmarks_and_frames(video_path):
    """Extract pose landmarks + keep raw frames for later video splitting."""
    cap = cv2.VideoCapture(video_path)
    sequence, frames = [], []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        if results.pose_landmarks:
            frame_landmarks = []
            for lm in results.pose_landmarks.landmark:
                frame_landmarks.extend([lm.x, lm.y, lm.z])
            sequence.append(frame_landmarks)
        else:
            # Append dummy frame to keep frame sync
            sequence.append([0]*99)
    cap.release()
    return np.array(sequence), frames

def get_motion_signal(sequence):
    """Choose the best joint (nose, shoulder, or hip) for motion tracking."""
    joints = [0, 11, 23]  # nose, left_shoulder, left_hip
    motion_signals = []
    for j in joints:
        y = sequence[:, j * 3 + 1]
        y_smooth = np.convolve(y, np.ones(7)/7, mode='same')
        motion_signals.append(y_smooth)
    ranges = [np.ptp(sig) for sig in motion_signals]
    best_idx = np.argmax(ranges)
    return motion_signals[best_idx]

def split_reps(sequence, signal, frames, save_dir, basename, fps):
    """Split video into *both* up→down→up and down→up→down reps."""
    peaks, _ = find_peaks(signal, distance=15, prominence=0.01)
    valleys, _ = find_peaks(-signal, distance=15, prominence=0.01)

    # Merge and sort all extrema
    extrema = np.sort(np.concatenate([peaks, valleys]))

    rep_count = 0

    # Loop through consecutive triples of extrema (A → B → C)
    for i in range(len(extrema) - 2):
        a, b, c = extrema[i], extrema[i + 1], extrema[i + 2]

        # Basic validity: ensure frames in between
        if c - a < 15:
            continue

        # Check the pattern type:
        # Peak-Valley-Peak (up-down-up) or Valley-Peak-Valley (down-up-down)
        if (signal[a] > signal[b] and signal[c] > signal[b]) or \
           (signal[a] < signal[b] and signal[c] < signal[b]):

            rep_seq = sequence[a:c]
            rep_frames = frames[a:c]
            rep_count += 1

            # Save numpy data
            npy_path = os.path.join(save_dir, f"{basename}_rep{rep_count}.npy")
            np.save(npy_path, rep_seq)

            # Save video segment
            h, w, _ = rep_frames[0].shape
            out_path = os.path.join(save_dir, f"{basename}_rep{rep_count}.mp4")
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
            for f in rep_frames:
                out.write(f)
            out.release()

    return rep_count

for filename in os.listdir(input_dir):
    if not filename.endswith(".mp4"):
        continue

    print("Processing:", filename)
    video_path = os.path.join(input_dir, filename)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    sequence, frames = extract_landmarks_and_frames(video_path)
    if sequence.size == 0:
        print("  ⚠️ No pose detected, skipping:", filename)
        continue

    signal = get_motion_signal(sequence)

    name = os.path.splitext(filename)[0]
    save_dir = os.path.join(output_root, name)
    os.makedirs(save_dir, exist_ok=True)

    reps_found = split_reps(sequence, signal, frames, save_dir, name, fps)
    print(f"  ✅ {reps_found} full reps saved to {save_dir}")

pose.close()
print("🎉 All videos processed successfully.")


Processing: Wrong Pushup Vid 1.mp4
  ✅ 19 full reps saved to split_reps_full_Wrong/Wrong Pushup Vid 1
Processing: Wrong Pushup vid 2.mp4
  ✅ 48 full reps saved to split_reps_full_Wrong/Wrong Pushup vid 2
Processing: Wrong Pushup Vid 3.mp4
  ✅ 9 full reps saved to split_reps_full_Wrong/Wrong Pushup Vid 3
Processing: Wrong Pushup VId 4.mp4
  ✅ 21 full reps saved to split_reps_full_Wrong/Wrong Pushup VId 4
🎉 All videos processed successfully.


Putting all into one

In [8]:
import numpy as np
import os, glob

base_dir = "split_reps_full_thistime/"
output_X = "all_reps_X.npy"
output_y = "all_reps_y.npy"

target_frames = 150  # length per rep

files = glob.glob(os.path.join(base_dir, "**/*.npy"), recursive=True)
print(f"Found {len(files)} files")

X = []
y = []
max_features = 0

# First pass: find max feature dimension
for f in files:
    data = np.load(f, allow_pickle=True)
    if data.ndim == 3:
        data = data[0] if data.shape[0] == 1 else data
    if data.ndim == 2:
        max_features = max(max_features, data.shape[1])
print(f"Detected max feature count: {max_features}")

# Second pass: normalize all to same shape
for f in files:
    data = np.load(f, allow_pickle=True)
    if data.ndim == 3:
        data = data[0] if data.shape[0] == 1 else data
    if data.ndim != 2:
        print(f"⚠️ Skipping invalid: {f}, shape={data.shape}")
        continue

    frames, feats = data.shape

    # pad features if needed
    if feats < max_features:
        feat_pad = np.zeros((frames, max_features - feats))
        data = np.hstack((data, feat_pad))
    elif feats > max_features:
        data = data[:, :max_features]

    # pad/crop frames
    if frames < target_frames:
        frame_pad = np.zeros((target_frames - frames, max_features))
        data = np.vstack((data, frame_pad))
    elif frames > target_frames:
        data = data[:target_frames, :]

    X.append(data)

    # auto-label based on filename
    if "good" in f.lower():
        y.append(1)
    elif "bad" in f.lower():
        y.append(0)

X = np.array(X)
print("✅ Combined dataset shape:", X.shape)
np.save(output_X, X)
print(f"💾 Saved {output_X}")

if y:
    y = np.array(y)
    np.save(output_y, y)
    print(f"💾 Saved {output_y} (labels 0=bad,1=good)")


Found 84 files
Detected max feature count: 99
✅ Combined dataset shape: (84, 150, 99)
💾 Saved all_reps_X.npy


In [3]:
%pip install opencv-python

     ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
     --------------------------------------- 0.0/39.0 MB 660.6 kB/s eta 0:00:59
     ---------------------------------------- 0.1/39.0 MB 1.3 MB/s eta 0:00:30
     ---------------------------------------- 0.2/39.0 MB 2.4 MB/s eta 0:00:17
      --------------------------------------- 0.5/39.0 MB 3.7 MB/s eta 0:00:11
      --------------------------------------- 0.9/39.0 MB 5.4 MB/s eta 0:00:08
     - -------------------------------------- 1.4/39.0 MB 6.3 MB/s eta 0:00:07
     - -------------------------------------- 1.8/39.0 MB 7.3 MB/s eta 0:00:06
     -- ------------------------------------- 2.2/39.0 MB 7.5 MB/s eta 0:00:05
     -- ------------------------------------- 2.7/39.0 MB 8.1 MB/s eta 0:00:05
     --- ------------------------------------ 3.1/39.0 MB 8.7 MB/s eta 0:00:05
     --- ------------------------------------ 3.6/39.0 MB 8.9 MB/s eta 0:00:04
     ---- ----------------------------------- 4.1/39.0 MB 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 6.33.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Mixing

In [13]:
import numpy as np

# ======== CONFIG ========
old_file = "Training Data VSP\labels\correct.npy"          # old 22-landmark data
new_file = "all_reps_X.npy"       # new 33-landmark data
merged_output = "merged_X.npy"    # output combined file
# =========================

print("🔹 Loading files...")
old_data = np.load(old_file, allow_pickle=True)
new_data = np.load(new_file, allow_pickle=True)

# ======== Handle old data ========
if old_data.ndim == 3 and old_data.shape[-1] == 66:
    # (samples, frames, 66) → reshape to (samples, frames, 22, 3)
    old_data = old_data.reshape(old_data.shape[0], old_data.shape[1], 22, 3)
elif old_data.ndim == 2 and old_data.shape[-1] == 66:
    # (frames, 66)
    old_data = old_data.reshape(1, old_data.shape[0], 22, 3)
else:
    raise ValueError(f"Unexpected old data shape: {old_data.shape}")

# ======== Handle new data ========
if new_data.ndim == 3 and new_data.shape[-1] == 99:
    new_data = new_data.reshape(new_data.shape[0], new_data.shape[1], 33, 3)
elif new_data.ndim == 2 and new_data.shape[-1] == 99:
    new_data = new_data.reshape(1, new_data.shape[0], 33, 3)
else:
    raise ValueError(f"Unexpected new data shape: {new_data.shape}")

print(f"Old data shape (before conversion): {old_data.shape}")
print(f"New data shape: {new_data.shape}")

# ======== CONVERT OLD 22 → 33 landmarks ========
print("🔹 Converting old data to 33-landmark format...")
num_samples, frames, _, _ = old_data.shape
converted_old = np.zeros((num_samples, frames, 33, 3), dtype=np.float32)
converted_old[:, :, :22, :] = old_data  # copy old joints, pad remaining 11 with zeros

# Flatten both into (samples, frames, 99)
converted_old_flat = converted_old.reshape(num_samples, frames, 99)
new_data_flat = new_data.reshape(new_data.shape[0], new_data.shape[1], 99)

# ======== Combine datasets ========
print("🔹 Merging both datasets...")
merged = np.concatenate([converted_old_flat, new_data_flat], axis=0)
np.save(merged_output, merged)

print(f"✅ Merged dataset saved to: {merged_output}")
print(f"   Total samples: {merged.shape[0]}")
print(f"   Frames per sample: {merged.shape[1]}")
print(f"   Features per frame: {merged.shape[2]}")
print("🎉 Conversion + Merge Complete!")


🔹 Loading files...
Old data shape (before conversion): (50, 150, 22, 3)
New data shape: (84, 150, 33, 3)
🔹 Converting old data to 33-landmark format...
🔹 Merging both datasets...
✅ Merged dataset saved to: merged_X.npy
   Total samples: 134
   Frames per sample: 150
   Features per frame: 99
🎉 Conversion + Merge Complete!
